In [2]:
import datetime as dt
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import time

In [3]:
# -------------------------------------------
# Modify the holiday class to 
# 1. Only accept Datetime objects for date.
# 2. You may need to add additional functions
# 3. You may drop the init if you are using @dataclasses
# --------------------------------------------
class Holiday:
    def __init__(self, name, date):
        self.name = name
        self.date = date

    def as_dict(self):
        return {'name': self.name, 'date': str(self.date.date())}

    def __str__ (self):
        # String output
        # Holiday output when printed.
        return f"{self.name} ({self.date.date()})"

    def __eq__ (self, other):
        return (self.name == other.name and self.date == other.date)

In [6]:
# # -------------------------------------------
# # The HolidayList class acts as a wrapper and container
# # For the list of holidays
# # Each method has pseudo-code instructions
# # --------------------------------------------
@dataclass
class HolidayList:
    def __init__(self):
        self.innerHolidays = []
        self.changes = False
        self.running = True

    def addHoliday(self):
        print("\nAdd a Holiday")
        print("================")     
        self.changes = True
        holiday = self.getHolidayName()
        date = self.getValidDate()
        self.createHoliday(Holiday(holiday, date))     
        
    def createHoliday(self, holidayObj):
        # Make sure holidayObj is an Holiday Object by checking the type
        # Use innerHolidays.append(holidayObj) to add holiday            
        # print to the user that you added a holiday
        if isinstance(holidayObj, Holiday):
            if holidayObj not in self.innerHolidays:
                self.innerHolidays.append(holidayObj)
        else:
            print(f"{holidayObj} is not a Holiday object.")
            
    def findHoliday(self, HolidayName, Date):
        # Find Holiday in innerHolidays
        # Return Holiday
        holiday = Holiday(HolidayName, Date)
        if holiday in self.innerHolidays:
            return holiday
        else:
            print(f"\n{HolidayName} ({Date.date()}) not in list.")
            
    def removeHoliday(self):
        # Find Holiday in innerHolidays by searching the name and date combination.
        # remove the Holiday from innerHolidays
        # inform user you deleted the holiday
        print(f"\nRemove Holiday from List")
        print("================") 
        self.changes = True
        name = self.getHolidayName()
        date = self.getValidDate()
        self.reallyRemoveHoliday(name, date)
            
    def reallyRemoveHoliday(self, HolidayName, Date):
        holiday = self.findHoliday(HolidayName, Date)       
        if holiday is None:
            return
        else:
            self.innerHolidays.remove(holiday)
            print(f"\n{holiday} has been removed from the list.")
            
    def read_json(self, filelocation):
        # Read in things from json file location
        # Use addHoliday function to add holidays to inner list.
        try:
            with open(filelocation, 'r') as j:
                holidays = json.loads(j.read())        
            for holiday in holidays['holidays']:
                date = holiday['date']
                holiday_name = holiday['name']
                new_date = dt.datetime.strptime(date, '%Y-%m-%d')
                self.createHoliday(Holiday(holiday_name, new_date))
        except:
            return
            
    def save_to_json(self):
        # Write out json file to selected file.
        print(f"\nSaving Holidays")
        print("================")
        while 1:
            choice = input(f"\nAre you sure you want to save your changes? Y/N:").upper()
            if choice == "Y":
                self.saveJSON('holidays.json')
                break;
            elif choice == "N":
                print(f"\nSave Cancelled.")
                break;
            else:
                print(f"\nPlease enter Y/N")
                
    def saveJSON(self, filelocation):
        save = [holiday.as_dict() for holiday in self.innerHolidays]
        f = dict()
        f['holidays'] = list(save)
        holidays_json = json.dumps(f)
        with open(filelocation, 'w') as j_file:
            j_file.write(holidays_json)
        self.changes = False
        print(f"\nSave Complete.")
        
    def getHTML(self, url):
        response = requests.get(url)
        return response.text
    
    def scrapeHolidays(self):
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        # Remember, 2 previous years, current year, and 2  years into the future. You can scrape multiple years 
        # by adding year to the timeanddate URL. For example https://www.timeanddate.com/holidays/us/2022
        # Check to see if name and date of holiday is in innerHolidays array
        # Add non-duplicates to innerHolidays
        # Handle any exceptions.
        self.changes = True
        for x in range(dt.date.today().year-2,dt.date.today().year+3):
            html = self.getHTML(f"https://www.timeanddate.com/holidays/us/{x}")
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find('tbody')       
            for row in table:
                holiday_name_tag = row.find('a')                
                try:
                    date_tag = row['data-date']
                except KeyError:
                    continue                        
                if holiday_name_tag is None:
                    continue
                else:
                    name = holiday_name_tag.text
                    date = dt.datetime.utcfromtimestamp(int(date_tag)/1000).strftime('%Y-%m-%d')
                    new_date = dt.datetime.strptime(date, '%Y-%m-%d')
                    self.createHoliday(Holiday(name, new_date))

            
    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)
    
    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # return your holidays
        holidays = list(filter(lambda x: x.date.isocalendar().week == week_number and x.date.isocalendar().year == year, self.innerHolidays))
        return holidays

    def viewHolidays(self):
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        # * Remember to use the holiday __str__ method.
        print("\nView Holidays")
        print("=================")
        current_year = dt.date.today().year
        while 1:
            year_input = input(f"\nChoose year - {current_year-2}-{current_year+2} or leave blank to select current year: ")
            if not year_input:
                year_input = str(current_year)
            if year_input.isdigit() and int(year_input) in range(current_year - 2, current_year + 3):                
                while 1:
                    week_input = input(f"\nChoose week? 1-52 or leave blank to select current week: ")
                    if not week_input:
                        self.viewCurrentWeek()
                        break;
                    elif week_input.isdigit() and int(week_input) > 0 and int(week_input) <= 52:                        
                        self.weekHolidays(int(year_input), int(week_input))
                        break;
                    else:
                        print(f"\nPlease enter a valid choice.")
                else:
                    continue
                break;
            else:
                print(f"\nPlease enter a valid choice.")
                
    def printList(self):
        for h in self.innerHolidays:
            print(h)    
            
    def weekHolidays(self, year, week):
        holidays = self.filter_holidays_by_week(year, week)
        print(f"\nHolidays in {year}, week {week}:")
        list(map(lambda x: print(x), holidays))

    def getWeather(self, weekNum):
        # Convert weekNum to range between two days
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        # Format weather information and return weather string.
        api_key = "6ef8bee4b8b4c7a671a7e59dd96f9a33"
        base_url = "http://api.openweathermap.org/data/2.5/weather?"
        city_name = "Milwaukee"
        complete_url = base_url + "appid=" + 'd850f7f52bf19300a9eb4b0aa6b80f0d' + "&q=" + city_name
        response = requests.get(complete_url)
        x = response.json()
        print(x)

    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        # Use your filter_holidays_by_week function to get the list of holidays 
        # for the current week/year
        # Ask user if they want to get the weather
        # If yes, use your getWeather function and display results
        current_year = dt.date.today().year
        current_week = dt.date.today().isocalendar().week
        current_week_holidays = self.filter_holidays_by_week(current_year, current_week)
        while 1: 
            choice = input("Do you want this week's weather? Y/N").upper()
            if choice == "Y":
                weather = self.getWeather(current_week)
                #Multiple getWeathers for all days of week here
            elif choice == "N":
                self.weekHolidays(current_year, current_week)
                break;
            else:
                print(f"\nPlease enter a valid choice.")    
        
    def getHolidayName(self):
        while 1:
            holiday = input("New Holdiay Name: ")
            if not holiday:
                print("Please enter a holiday name")
            else:
                return holiday
        
    def getValidDate(self):
        while 1:      
            date = input("Date: ")
            date_string = date
            date_format = '%Y-%m-%d'
            try:
                date_obj = dt.datetime.strptime(date_string, date_format)
                # date = date_obj.strftime('%Y-%m-%d')
                return date_obj
            except ValueError:
                print(f"\nPlease enter the date as 'YYYY-MM-DD'.")
                
    def removeHolidayTxt(self):
        print(f"\nRemove Holiday from List")
        print("====================") 
        self.changes = True
        name = self.getHolidayName()
        date = self.getValidDate()
        self.removeHoliday(name, date)            

    def exit(self):
        print("\nExit")
        print("================")
        st = ""
        while 1:
            if self.changes:
                st = "\nYOU HAVE UNSAVED CHANGES!\n"           
            choice = input(f"\nAre you sure you want to exit? {st}Y/N").upper()
            if choice == "Y":
                self.running = False
                print("Goodbye!")
                return
            elif choice == "N":
                print("Cancelled.")
                return
            else:
                print(f"\nPlease enter a valid choice.")                
                
def main():
# Large Pseudo Code steps
# -------------------------------------
# 1. Initialize HolidayList Object
# 2. Load JSON file via HolidayList read_json function
# 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
# 4. Create while loop for user to keep adding or working with the Calender
# holidayList.printList()
# 4. Display User Menu (Print the menu)
# 5. Take user input for their action based on Menu and check the user input for errors
# 6. Run appropriate method from the HolidayList object depending on what the user input is
# 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 
    holidayList = HolidayList()
    holidayList.read_json('holidays.json')
    holidayList.scrapeHolidays()
    while holidayList.running:
        print(f"\nHoliday Management")
        print(f"===================")
        print(f"There are {holidayList.numHolidays()} holidays stored in the system.")
        print(f"\nHoliday Menu")
        print(f"================")
        print(f"1) Add Holiday")
        print(f"2) Remove Holiday")
        print(f"3) Save List")
        print(f"4) View Holidays")
        print(f"5) Exit Program") 
        while 1:
            user_choice = input("Please enter menu choice (1-5)")
            if user_choice.isdigit() and int(user_choice) > 0 and int(user_choice) <= 5:
                choice = int(user_choice)
                break;
            else:
                print(f"\nPlease enter a valid choice.")      
        menu = [holidayList.addHoliday, holidayList.removeHoliday, holidayList.save_to_json, holidayList.viewHolidays, holidayList.exit]
        menu[choice-1]()

if __name__ == "__main__":
    main();


# Additional Hints:
# ---------------------------------------------
# You may need additional helper functions both in and out of the classes, add functions as you need to.
#
# No one function should be more then 50 lines of code, if you need more then 50 lines of code
# excluding comments, break the function into multiple functions.
#
# You can store your raw menu text, and other blocks of texts as raw text files 
# and use placeholder values with the format option.
# Example:
# In the file test.txt is "My name is {fname}, I'm {age}"
# Then you later can read the file into a string "filetxt"
# and substitute the placeholders 
# for example: filetxt.format(fname = "John", age = 36)
# This will make your code far more readable, by seperating text from code.


Holiday Management
There are 2559 holidays stored in the system.

Holiday Menu
1) Add Holiday
2) Remove Holiday
3) Save List
4) View Holidays
5) Exit Program


Please enter menu choice (1-5) 4



View Holidays



Choose year - 2020-2024 or leave blank to select current year:  2022

Choose week? 1-52 or leave blank to select current week:  y



Please enter a valid choice.



Choose week? 1-52 or leave blank to select current week:  
Do you want this week's weather? Y/N y


{'coord': {'lon': -87.9668, 'lat': 43.0006}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 259.12, 'feels_like': 252.12, 'temp_min': 256.8, 'temp_max': 261.51, 'pressure': 1032, 'humidity': 42}, 'visibility': 10000, 'wind': {'speed': 3.58, 'deg': 359, 'gust': 6.26}, 'clouds': {'all': 20}, 'dt': 1641846239, 'sys': {'type': 2, 'id': 2034123, 'country': 'US', 'sunrise': 1641820929, 'sunset': 1641854176}, 'timezone': -21600, 'id': 5263058, 'name': 'Milwaukee', 'cod': 200}


Do you want this week's weather? Y/N n



Holidays in 2022, week 2:
Stephen Foster Memorial Day (2022-01-13)
Orthodox New Year (2022-01-14)
World Religion Day (2022-01-16)

Holiday Management
There are 2559 holidays stored in the system.

Holiday Menu
1) Add Holiday
2) Remove Holiday
3) Save List
4) View Holidays
5) Exit Program


KeyboardInterrupt: Interrupted by user

<Response [200]>
